# Análise Exploratória de Dados

In [51]:
# importando bibliotecas
import os # manipulação de pastas
import warnings # exibe avisos
import numpy as np # manipulação numérica e matricial
import pandas as pd # manipulação de tabelas
import seaborn as sns # geração de gráficos
import matplotlib.pyplot as plt # geração de gráficos

warnings.filterwarnings('ignore')

In [52]:
# importando conjunto de dados
df = pd.read_csv('../data/raw/google_ads_data.csv')

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ad_ID            2600 non-null   object 
 1   Campaign_Name    2600 non-null   object 
 2   Clicks           2488 non-null   float64
 3   Impressions      2546 non-null   float64
 4   Cost             2503 non-null   object 
 5   Leads            2552 non-null   float64
 6   Conversions      2526 non-null   float64
 7   Conversion Rate  1974 non-null   float64
 8   Sale_Amount      2461 non-null   object 
 9   Ad_Date          2600 non-null   object 
 10  Location         2600 non-null   object 
 11  Device           2600 non-null   object 
 12  Keyword          2600 non-null   object 
dtypes: float64(5), object(8)
memory usage: 264.2+ KB


----

## 1. Tratamento dos Dados

In [54]:
df.head()

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Location,Device,Keyword
0,A1000,DataAnalyticsCourse,104.0,4498.0,$231.88,14.0,7.0,0.058,$1892,2024-11-16,hyderabad,desktop,learn data analytics
1,A1001,DataAnalyticsCourse,173.0,5107.0,$216.84,10.0,8.0,0.046,$1679,20-11-2024,hyderabad,mobile,data analytics course
2,A1002,Data Anlytics Corse,90.0,4544.0,$203.66,26.0,9.0,NaN,$1624,2024/11/16,hyderabad,Desktop,data analitics online
3,A1003,Data Analytcis Course,142.0,3185.0,$237.66,17.0,6.0,NaN,$1225,2024-11-26,HYDERABAD,tablet,data anaytics training
4,A1004,Data Analytics Corse,156.0,3361.0,$195.9,30.0,8.0,NaN,$1091,2024-11-22,hyderabad,desktop,online data analytic


In [55]:
# visualização das categorias
for col in df.columns:
    print(col.upper())
    print(df[col].value_counts())
    print('_'*50)

AD_ID
A1000    1
A2746    1
A2728    1
A2729    1
A2730    1
        ..
A1868    1
A1869    1
A1870    1
A1871    1
A3599    1
Name: Ad_ID, Length: 2600, dtype: int64
__________________________________________________
CAMPAIGN_NAME
Data Analytcis Course    680
Data Analytics Corse     647
DataAnalyticsCourse      637
Data Anlytics Corse      636
Name: Campaign_Name, dtype: int64
__________________________________________________
CLICKS
127.0    34
148.0    32
89.0     29
146.0    28
151.0    28
         ..
107.0    13
129.0    13
159.0    12
161.0    12
158.0    11
Name: Clicks, Length: 120, dtype: int64
__________________________________________________
IMPRESSIONS
4653.0    6
4441.0    5
5509.0    5
5545.0    5
3948.0    5
         ..
4224.0    1
3579.0    1
3188.0    1
5493.0    1
5453.0    1
Name: Impressions, Length: 1702, dtype: int64
__________________________________________________
COST
$246.3     4
$218.69    4
$219.15    4
$245.13    4
$249.33    4
          ..
$223.28    1


**Observações:**
- Notamos que na variável `Campaing_Name` as 4 campanhas possuem o mesmo nome, porém com escrituras diferentes. Apesar do mesmo nome, não devem ser a mesma campanha e sim campanhas distintas, afinal nosso trabalho é fazer a análise de qual tem a melhor perfomance. Dito isso, a forma que optarei para corrigir será substituindo os nomes por "Campanha 1", "Campanha 2" e sucessivamente.
- Em `Location` vemos que a localização é no mesmo lugar em todas as campanhas: Hyderabad (Capital e maior cidade do estado indiano de Tengala). Sabendo disso, ao invés de tratar os caracteres da coluna, podemos simplesmente remove-la já que se aplica a todos os registros e não tem variância, não adiciona informação para a análise de performance, além de economizar memória e torna o conjunto de dados mais limpo.
- O tipo de dispositivo em `Device` está com valores diferentes para cada categoria, poderemos resolver corrigindo os caracteres e agregando-os.
- As keywords parecem não haver um ádrão definido, porém é uma estratégia de marketing testar combinações diferentes de palavras-chave, para ver se há impacto sobre as campanhas. O que podemos fazer futuramente antes de treinar um modelo de machine learning, é separar cada uma das palavras em variáveis diferentes para ver se uma tem mais impacto que outra.


**Removendo a coluna Location**

In [56]:
df.drop(columns='Location', axis=1, inplace=True)

**Tratando o Nome da Campanha**

In [57]:
# Listando os valores únicos
campanha_unic = df['Campaign_Name'].unique()

# criando um dicionário de mapeamento
mapa = {
    campanha_unic[0]: 'campanha_1',
    campanha_unic[1]: 'campanha_2',
    campanha_unic[2]: 'campanha_3',
    campanha_unic[3]: 'campanha_4'
}

# aplicando a substituição
df['Campaign_Name'] = df['Campaign_Name'].map(mapa)

df['Campaign_Name'].value_counts()

campanha_3    680
campanha_4    647
campanha_1    637
campanha_2    636
Name: Campaign_Name, dtype: int64

**Tratando os Tipos de Dispsitivos**

In [58]:
# padroniza os caracteres em minusculo
df['Device'] = df['Device'].apply(lambda x: x.lower())

In [59]:
df.Device.value_counts()

desktop    888
mobile     878
tablet     834
Name: Device, dtype: int64

**Tratando Keywords**

In [60]:
# substitui os espaçamentos em cadacategoria por "_"
df['Keyword'] = df['Keyword'].apply(lambda x: x.replace(' ', '_'))

 **Tratando Inconcistências em Variáveis Financeiras**

In [61]:
# lista as variáveis financeiras
fin = ['Cost', 'Sale_Amount']

# itera sobre cada uma das variáveis
for col in df[fin]:
    # remove caractere "$"
    df[col] = df[col].str.replace('$', '', regex=False)
    # formata os valores numéricos para o tipo float
    df[col] = df[col].astype(float)

**Corrigindo variável Data**

- Algumas datas estão no formato YYYY-MM-DD, outras no formato DD-MM-YYYY.
- Precisamos primeiro padronizar para um único formato e depois converter para o `datetime` do Pandas.

In [62]:
# tentando converter diretamente todos os valores
df['Ad_Date'] = pd.to_datetime(df['Ad_Date'], errors='coerce', dayfirst=False)

# identificando datas que falharam
erros = df[df['Ad_Date'].isna()]

# para as datas que fallharam, tentar converter novamente
df.loc[df['Ad_Date'].isna(), 'Ad_Date'] = pd.to_datetime(
    df.loc[df['Ad_Date'].isna(), 'Ad_Date'], errors='coerce', dayfirst=True
)

# conferindo se ainda há falhas
df[df['Ad_Date'].isna()]

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Device,Keyword


**Tratando Valores Nulos**

In [63]:
df.isnull().sum()

Ad_ID                0
Campaign_Name        0
Clicks             112
Impressions         54
Cost                97
Leads               48
Conversions         74
Conversion Rate    626
Sale_Amount        139
Ad_Date              0
Device               0
Keyword              0
dtype: int64

**Observações:**
- Possuímos valores nulos em: Cliques, impressões, custo, leads, conversões, taxa de conversão, valor da venda.
- 112 anúncios estão com dados nulos em cliques, faz sentido? Pode ser que esses valores ausentes não sejam por inconsistência, e sim por que não geraram cliques mesmo. Podemos substituir os valores nulos por "0" e posteriormente fazer uma análise das campanhas que não geraram cliques e abstrair algum insight.
- Temos também 54 anúncios com impressões nulas, por quê? Seria devido ao fato de que realmente não houveram as impressões ou apenas inconcistência? Faz sentido para o negócio? Assim como em Cliques, iremos substituir por "0" para uma análise futura.
- No caso dos nulos em Custo do anúncio, seria também o fator inconsistência? Pode ser anúncios em fase de espera de aprovação, problemas de pagamento ou até mesmo não ter gerado nenhum resultado mensurável.
- As mesmas questões acima se aplicam para a variável de Conversão.
- 626 nulos em Taxa de Conversão aparenta ser estranho, principalmente em um conjunto de dados com apenas 2600 registros. Aqui pode haver de fato uma inconcistência nos dados.
- Em Sale_Amount temos 139 nulos, mas como essa variável determina a receita gerada após a conversão, podemos supor que as conversões em específico não foram de vendas, e talvez Leads.

Antes de remover qualquer valor desses, iremos conferir se de fato é uma inconcistência ou apenas falta de tratamento.

----

**Preenchendo cliques e impressões nulas**

In [64]:
df['Clicks'].fillna(0, inplace=True)
df['Impressions'].fillna(0, inplace=True)

**Tratando os anúncios com custos nulos**

Para esta variável uma possível interpretação para os custos de anúncio estarem vazios é que ainda não foram colocados em andamento, logo faz sentido não ter gerado custos. E se a campanha não foi anunciada, não tratá informações para podermos analisar. Antes de remover os nulos, vamos conferir  se de fato não trouxeram informações.

In [65]:
df_custo_nulo = df.loc[df['Cost'].isna()]
df_custo_nulo

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Device,Keyword
8,A1008,campanha_4,113.0,5434.0,NaN,27.0,4.0,0.058,1362.0,2024-11-24,tablet,data_anaytics_training
16,A1016,campanha_3,193.0,5159.0,NaN,15.0,9.0,0.047,1614.0,2024-11-10,mobile,learn_data_analytics
19,A1019,campanha_3,145.0,5278.0,NaN,25.0,6.0,0.041,1516.0,2024-11-05,desktop,data_analytics_course
24,A1024,campanha_1,87.0,3718.0,NaN,12.0,3.0,0.034,1652.0,2024-11-20,tablet,learn_data_analytics
37,A1037,campanha_3,123.0,5131.0,NaN,19.0,5.0,0.041,1307.0,2024-09-11,mobile,learn_data_analytics
...,...,...,...,...,...,...,...,...,...,...,...,...
2497,A3497,campanha_2,135.0,4611.0,NaN,17.0,7.0,NaN,1697.0,2024-11-14,desktop,online_data_analytic
2522,A3522,campanha_4,127.0,3682.0,NaN,21.0,9.0,0.071,1384.0,2024-11-22,tablet,data_anaytics_training
2528,A3528,campanha_1,132.0,5616.0,NaN,28.0,7.0,0.053,1491.0,2024-11-26,desktop,data_analitics_online
2581,A3581,campanha_3,0.0,5297.0,NaN,27.0,9.0,NaN,1268.0,2024-11-23,desktop,data_analytics_course


- Como podemos observar, a suposição que tive anteriormente parece estar equivocada uma vez que ainda há cliques, impressões, leads etc, em campanhas com custo nulo. Poderia ser inconsistência na captura dos dados ou algum outro fator que justifique?

In [66]:
df_custo_nulo.describe(include='all')

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Device,Keyword
count,97,97,97.000000,97.000000,0.0,95.000000,95.000000,69.000000,95.000000,97,97,97
unique,97,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,3,6
top,A1008,campanha_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-14 00:00:00,tablet,data_analitics_online
freq,1,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,37,21
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-11 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-11 00:00:00,NaN,NaN
mean,NaN,NaN,130.515464,4423.319588,NaN,20.031579,6.452632,0.047000,1472.631579,NaN,NaN,NaN
std,NaN,NaN,46.347221,1215.015051,NaN,5.970589,2.234966,0.017382,268.820515,NaN,NaN,NaN
min,NaN,NaN,0.000000,0.000000,NaN,10.000000,3.000000,0.016000,1016.000000,NaN,NaN,NaN
25%,NaN,NaN,112.000000,3718.000000,NaN,15.000000,5.000000,0.035000,1259.500000,NaN,NaN,NaN


In [67]:
# criando dataframe temporário
_ = df.copy()

# criando coluna de mês
_['mes'] = pd.to_datetime(_['Ad_Date']).dt.month

# agrupando os custos nulos por mês
_.groupby('mes')['Cost'].apply(lambda x: x.isna().sum())

mes
1      0
2      1
3      1
4      0
5      0
6      2
7      0
8      2
9      2
10     1
11    86
12     2
Name: Cost, dtype: int64

- Parece que nos mês 11 (Novembro) ocorreu algo que aumentou significativamente a quantidade de campanhas com custos nulos. Em um caso real iria reportar o ocorrido ao time de negócios para tentar identificar a causa, porém nesta situação irei continuar a análise e definir o melhor modo para imputar ou remover os dados nulos.

In [68]:
# dristribuição de custos no mês de novembro
_['Cost'].loc[_['mes'] == 11].describe()

count    2226.000000
mean      214.936271
std        20.296684
min       180.010000
25%       197.280000
50%       214.785000
75%       232.865000
max       249.890000
Name: Cost, dtype: float64

- Para os nulos no mês de novembro, irei imputar os dados como a média deste mês, visto que a distribuição da média e da mediana estão próximas e isso sugere uma distribuição normal, logo os valores imputados não serão tão distantes do valor real.
- Estou optando fazer a imputação dos nulos para evitar revmove-los e com isso perder informação do conjunto de dados.

In [ ]:
# armazenando valor de custo médio apenas do mês de novembro
media_custo_11 = _.loc[_['mes'] == 11, 'Cost'].mean() 

# substituindo apenas os nulos do mês de novembro no dataset original
df.loc[(_.Cost.isna()) & (_.mes == 11), 'Cost'] = media_custo_11

In [ ]:
# soma de nulos restante nos custos 
df.Cost.isna().sum()

11

- Para esses 11 registros farei uma imputação mais simples, utilizando o método Forward Fill, que preenche os nulos baseado nos valores das linhas acima dele.

In [74]:
# preenchendo nulos com base na linha de cima
df['Cost'].fillna(method='ffill', inplace=True)

----

**Analisando Conversões nulas**

In [ ]:
# crinaco dópia temporária do dataframe
_ = df.copy()

# filtrando onde os dados são nulos em Converssões
_ = _.loc[_['Conversions'].isna()]

_

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Device,Keyword
148,A1148,campanha_4,180.0,4326.0,221.67,13.0,NaN,NaN,1427.0,2024-06-11,tablet,online_data_analytic
211,A1211,campanha_1,101.0,3241.0,183.34,24.0,NaN,NaN,1483.0,2024-10-11,tablet,data_anaytics_training
213,A1213,campanha_4,199.0,5734.0,247.99,18.0,NaN,NaN,1455.0,2024-11-11,desktop,data_analitics_online
229,A1229,campanha_2,165.0,4551.0,185.35,30.0,NaN,NaN,1662.0,2024-11-06,desktop,data_analitics_online
233,A1233,campanha_2,161.0,5977.0,193.27,24.0,NaN,NaN,1366.0,2024-11-13,desktop,online_data_analytic
...,...,...,...,...,...,...,...,...,...,...,...,...
2533,A3533,campanha_2,130.0,5340.0,221.95,27.0,NaN,NaN,1488.0,2024-11-02,tablet,data_analitics_online
2543,A3543,campanha_3,158.0,3766.0,222.89,27.0,NaN,0.056,1673.0,2024-11-22,mobile,learn_data_analytics
2560,A3560,campanha_2,170.0,3401.0,235.70,10.0,NaN,NaN,1047.0,2024-11-15,mobile,online_data_analytic
2566,A3566,campanha_3,160.0,5975.0,212.18,14.0,NaN,NaN,1507.0,2024-11-04,tablet,data_analytics_course
